In [1]:
from pymongo import MongoClient

In [2]:
client = MongoClient("mongodb://localhost:27017/")
db = client["customer_db"]

In [3]:
collections = {
    "request": db["request"],
    "error": db["error"],
    "reject": db["reject"],
    "response": db["response"],
}

In [4]:
collections

{'request': Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'customer_db'), 'request'),
 'error': Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'customer_db'), 'error'),
 'reject': Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'customer_db'), 'reject'),
 'response': Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'customer_db'), 'response')}

In [5]:
import json
data = {}
for collection_name in db.list_collection_names():
    collection = db[collection_name]
    documents = list(collection.find({}, {"_id": 0}))  # Exclude _id field if not needed
    data[collection_name] = documents

# Convert to JSON format (optional)
json_data = json.dumps(data, indent=4)



In [6]:
from pprint import pprint
pprint(json_data)

('{\n'
 '    "response": [\n'
 '        {\n'
 '            "cusno": 1,\n'
 '            "name": "Alice",\n'
 '            "cusid": "CUST001"\n'
 '        },\n'
 '        {\n'
 '            "cusno": 3,\n'
 '            "name": "Charlie",\n'
 '            "cusid": "CUST003"\n'
 '        },\n'
 '        {\n'
 '            "cusno": 5,\n'
 '            "name": "Eve",\n'
 '            "cusid": "CUST005"\n'
 '        }\n'
 '    ],\n'
 '    "reject": [\n'
 '        {\n'
 '            "cusno": 2,\n'
 '            "name": "Bob",\n'
 '            "cusid": "CUST002"\n'
 '        },\n'
 '        {\n'
 '            "cusno": 4,\n'
 '            "name": "David",\n'
 '            "cusid": "CUST004"\n'
 '        }\n'
 '    ],\n'
 '    "request": [\n'
 '        {\n'
 '            "cusno": 6,\n'
 '            "name": "Frank",\n'
 '            "cusid": "CUST006"\n'
 '        },\n'
 '        {\n'
 '            "cusno": 7,\n'
 '            "name": "Grace",\n'
 '            "cusid": "CUST007"\n'
 '        }\n

In [7]:
len(json_data)

1137

In [8]:
data=json.loads(json_data)
print(len(data))

4


In [9]:
print(data)

{'response': [{'cusno': 1, 'name': 'Alice', 'cusid': 'CUST001'}, {'cusno': 3, 'name': 'Charlie', 'cusid': 'CUST003'}, {'cusno': 5, 'name': 'Eve', 'cusid': 'CUST005'}], 'reject': [{'cusno': 2, 'name': 'Bob', 'cusid': 'CUST002'}, {'cusno': 4, 'name': 'David', 'cusid': 'CUST004'}], 'request': [{'cusno': 6, 'name': 'Frank', 'cusid': 'CUST006'}, {'cusno': 7, 'name': 'Grace', 'cusid': 'CUST007'}], 'error': [{'cusno': 0, 'name': 'Eve', 'cusid': 'CUST002'}, {'cusno': 0, 'name': 'John', 'cusid': 'CUST004'}, {'cusno': 0, 'name': 'Henry', 'cusid': 'CUST008'}]}


In [10]:
# from langchain.text_splitter import RecursiveCharacterTextSplitter
# text_splitter=RecursiveCharacterTextSplitter(chunk_size=200)
# data=text_splitter.split_text(json_data)
# print(len(data))

In [11]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# vector=embeddings.embed_query("hello world")
# vector[:5]

C:\Users\T. HARIKISHAN REDDY\AppData\Local\Temp\ipykernel_12576\3087897998.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
# parsed_data = json.loads(json_data)

# # Convert JSON data into LangChain Document objects
# documents = []
# for category, records in parsed_data.items():
#     for record in records:
#         # Structured metadata
#         metadata = {
#             "category": category,
#             "customer": record["name"],
#             "customer_id": record["cusid"],
#             "customer_no": record["cusno"]
#         }

#         # Natural language representation
#         page_content = (
#             f"Customer {record['name']} ({record['cusid']}) is in category '{category}' "
#             f"with customer number {record['cusno']}."
#         )

#         doc = Document(page_content=page_content, metadata=metadata)
#         documents.append(doc)

# # Output
# documents

In [12]:
from langchain.schema import Document
# Parse JSON
parsed_data = json.loads(json_data)

# Convert JSON data into LangChain Document objects
documents = []
for category, records in parsed_data.items():
    for record in records:
        text = f"Category: {category}, Customer: {record['name']}, ID: {record['cusid']}, No: {record['cusno']}"
        doc = Document(page_content=text, metadata={"category": category})
        documents.append(doc)

documents

[Document(metadata={'category': 'response'}, page_content='Category: response, Customer: Alice, ID: CUST001, No: 1'),
 Document(metadata={'category': 'response'}, page_content='Category: response, Customer: Charlie, ID: CUST003, No: 3'),
 Document(metadata={'category': 'response'}, page_content='Category: response, Customer: Eve, ID: CUST005, No: 5'),
 Document(metadata={'category': 'reject'}, page_content='Category: reject, Customer: Bob, ID: CUST002, No: 2'),
 Document(metadata={'category': 'reject'}, page_content='Category: reject, Customer: David, ID: CUST004, No: 4'),
 Document(metadata={'category': 'request'}, page_content='Category: request, Customer: Frank, ID: CUST006, No: 6'),
 Document(metadata={'category': 'request'}, page_content='Category: request, Customer: Grace, ID: CUST007, No: 7'),
 Document(metadata={'category': 'error'}, page_content='Category: error, Customer: Eve, ID: CUST002, No: 0'),
 Document(metadata={'category': 'error'}, page_content='Category: error, Custo

In [13]:
from langchain.vectorstores import FAISS
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# Initialize Hugging Face Embeddings
# embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS Vector Store
vectorstore = FAISS.from_documents(documents, embeddings)


In [46]:
vectorstore

In [24]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# query = "how many  customers got rejected?"
# retrieved_docs = retriever.invoke(query)  # Using invoke() instead of get_relevant_documents()

# # Print retrieved documents
# for doc in retrieved_docs:
#     print(doc.page_content)


In [15]:
print(retrieved_docs[0].page_content)
print(retrieved_docs[1].page_content)

Category: reject, Customer: David, ID: CUST004, No: 4
Category: reject, Customer: Bob, ID: CUST002, No: 2


In [16]:
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [19]:
from langchain.llms import HuggingFacePipeline

In [55]:
# Use Flan-T5-Small for text generation
model_id = "google/flan-t5-base"

text_generation_pipeline = pipeline(
    "text2text-generation",
    model=model_id,
    max_new_tokens=400
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)
prompt_template="""
<|system|>
You are an AI assistant who analyze json text data . Your goal is to analyze the provided context and generate insightful responses in natural language. Answer concisely and accurately using only the relevant information from the question.

{context}

{question}

Below are the instructions.
- Use only the provided context to answer.
- Provide clear, structured, and concise responses.

"""


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

C:\Users\T. HARIKISHAN REDDY\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\T. HARIKISHAN REDDY\.cache\huggingface\hub\models--google--flan-t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


In [56]:
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

In [57]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | llm_chain
question = "list all the customer data in response collection"

rag_chain.invoke(question)

'Use only the provided context to answer.'

In [54]:
query = "list all the customer data in response collection"
retrieved_docs = retriever.invoke(query)  # Using invoke() instead of get_relevant_documents()

# Print retrieved documents
for doc in retrieved_docs:
    print(doc.page_content)

Category: response, Customer: Alice, ID: CUST001, No: 1
Category: response, Customer: Eve, ID: CUST005, No: 5
Category: response, Customer: Charlie, ID: CUST003, No: 3
Category: request, Customer: Grace, ID: CUST007, No: 7
Category: request, Customer: Frank, ID: CUST006, No: 6
